In [5]:
# Preprocessing

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

data_dir = r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated"

import os
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\Cities.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\Conferences.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MConferenceTourneyGames.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MGameCities.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MMasseyOrdinals.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MNCAATourneyCompactResults.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MNCAATourneyDetailedResults.csv
C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MNCAATourneySeedRoundSlots.csv
C:\User

There is an abundance of data. Although not all of it might be used, it's still important to inspect each dataset. Let's start with the tournament teams, tournament seeding, and regular season results.

In [23]:
teams_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MTeams.csv")

print("Men's tournament teams")
print(teams_m.head())

teams_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WTeams.csv")

print("\nWomen's tournament teams")
print(teams_w.head())

seeding_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MNCAATourneySeeds.csv")

print("\nMen's tournament seeds")
print(seeding_m.head())

seeding_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WNCAATourneySeeds.csv")

print("\nWomen's tournament seeds")
print(seeding_w.head())

tournament_compact_results_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MNCAATourneyCompactResults.csv")

print("\nMen's Compact Tournament Results")
print(tournament_compact_results_m.head())

tournament_compact_results_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WNCAATourneyCompactResults.csv")

print("\nWomen's Compact Tournament Results")
print(tournament_compact_results_w.head())

tournament_detailed_results_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MNCAATourneyDetailedResults.csv")

print("\nMen's Detailed Tournament Results")
print(tournament_detailed_results_m.head())

tournament_detailed_results_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WNCAATourneyDetailedResults.csv")

print("\nWomen's Detailed Tournament Results")
print(tournament_detailed_results_w.head())

regular_season_compact_results_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MRegularSeasonCompactResults.csv")

print("\nMen's Regular Season Compact Results")
print(regular_season_compact_results_m.head())

regular_season_compact_results_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WRegularSeasonCompactResults.csv")

print("\nWomen's Regular Season Compact Results")
print(regular_season_compact_results_w.head())

regular_season_detailed_results_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MRegularSeasonDetailedResults.csv")

print("\nMen's Detailed Season Compact Results")
print(regular_season_detailed_results_m.head())

regular_season_detailed_results_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WRegularSeasonDetailedResults.csv")

print("\nMen's Detailed Season Compact Results")
print(regular_season_detailed_results_w.head())

conferences_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MTeamConferences.csv")

print("\nMen's Team Conferences")
print(conferences_m.head())

conferences_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WTeamConferences.csv")

print("\nWomen's Team Conferences")
print(conferences_w.head())

Men's tournament teams
   TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2025
1    1102    Air Force           1985          2025
2    1103        Akron           1985          2025
3    1104      Alabama           1985          2025
4    1105  Alabama A&M           2000          2025

Women's tournament teams
   TeamID     TeamName
0    3101  Abilene Chr
1    3102    Air Force
2    3103        Akron
3    3104      Alabama
4    3105  Alabama A&M

Men's tournament seeds
   Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374

Women's tournament seeds
   Season Seed  TeamID
0    1998  W01    3330
1    1998  W02    3163
2    1998  W03    3112
3    1998  W04    3301
4    1998  W05    3272

Men's Compact Tournament Results
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985     136     1116      63     1234      54    N      0
1    1985    

# Exploratory Data Analysis

In [24]:
seeding_by_season_m = seeding_m
seeding_by_season_w = seeding_w

print(seeding_by_season_m.head())
print(seeding_by_season_w.head())

   Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374
   Season Seed  TeamID
0    1998  W01    3330
1    1998  W02    3163
2    1998  W03    3112
3    1998  W04    3301
4    1998  W05    3272


# Feature Engineering

The next steps are to be able to extract the win loss records of each season of each team. We need to the margin of victory, strength of schedule, and simple rating system.

In [25]:
def margin_of_victory(df: pd.DataFrame) -> pd.DataFrame:
    # First calculate MOV for each game
    df_with_mov = df.copy()
    df_with_mov['MOV'] = df_with_mov['WScore'] - df_with_mov['LScore']

    # Create dataframe for winning teams
    winners = df_with_mov[['Season', 'WTeamID', 'MOV']].rename(columns={'WTeamID': 'TeamID'})
    winners['Win'] = 1
    winners['Loss'] = 0

    # Create dataframe for losing teams (negative MOV)
    losers = df_with_mov[['Season', 'LTeamID', 'MOV']].rename(columns={'LTeamID': 'TeamID'})
    losers['MOV'] = -losers['MOV']
    losers['Win'] = 0
    losers['Loss'] = 1

    # Combine both dataframes
    all_teams = pd.concat([winners, losers])

    # Group by Season and TeamID to get aggregate stats
    team_mov = all_teams.groupby(['Season', 'TeamID']).agg(
        avg_MOV=('MOV', 'mean'),
        total_MOV=('MOV', 'sum'),
        games_played=('MOV', 'count'),
        wins=('Win', 'sum'),
        losses=('Loss', 'sum')
    ).reset_index()

    # Calculate winning percentage
    team_mov['win_pct'] = team_mov['wins'] / team_mov['games_played']

    return team_mov

mov_m = margin_of_victory(regular_season_compact_results_m)
mov_w = margin_of_victory(regular_season_compact_results_w)

print(mov_m.head())
print(mov_w.head())    

   Season  TeamID   avg_MOV  total_MOV  games_played  wins  losses   win_pct
0    1985    1102 -5.791667       -139            24     5      19  0.208333
1    1985    1103 -3.043478        -70            23     9      14  0.391304
2    1985    1104  7.800000        234            30    21       9  0.700000
3    1985    1106 -3.791667        -91            24    10      14  0.416667
4    1985    1108  7.960000        199            25    19       6  0.760000
   Season  TeamID    avg_MOV  total_MOV  games_played  wins  losses   win_pct
0    1998    3102 -20.625000       -495            24     4      20  0.166667
1    1998    3103  -5.862069       -170            29    11      18  0.379310
2    1998    3104  13.433333        403            30    21       9  0.700000
3    1998    3106  -7.952381       -167            21     6      15  0.285714
4    1998    3108   1.304348         30            23    12      11  0.521739


In [26]:
def simple_rating_system(df: pd.DataFrame, mov: pd.DataFrame) -> pd.DataFrame:
    # The mov is passed in from the margin_of_victory function, so avg_MOV is already calculated
    srs = mov.copy()

    # We initialize the SRS to the average MOV
    srs['SRS'] = srs['avg_MOV']

    # We iteratively calculate the SRS until it converges
    for i in range(100):
        # Get opponents' SRS for games where team was winner
        winner_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                             left_on=['Season', 'WTeamID'], 
                             right_on=['Season', 'TeamID'], 
                             how='left')
        winner_sos = winner_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                    left_on=['Season', 'LTeamID'], 
                                    right_on=['Season', 'TeamID'], 
                                    how='left',
                                    suffixes=['_team', '_opp'])
        
        # Get opponents' SRS for games where team was loser
        loser_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                            left_on=['Season', 'LTeamID'], 
                            right_on=['Season', 'TeamID'], 
                            how='left')
        loser_sos = loser_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                   left_on=['Season', 'WTeamID'], 
                                   right_on=['Season', 'TeamID'], 
                                   how='left',
                                   suffixes=['_team', '_opp'])
        
        # Combine and calculate average opponent SRS
        winner_opponents = winner_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        loser_opponents = loser_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        
        all_opponents = pd.concat([winner_opponents, loser_opponents])
        sos = all_opponents.groupby(['Season', 'TeamID'])['SRS_opp'].mean().reset_index()
        sos.columns = ['Season', 'TeamID', 'SOS']

        # Update SRS
        srs = srs.merge(sos, on=['Season', 'TeamID'], how='left')
        srs['SRS'] = srs['avg_MOV'] + srs['SOS']
        srs.drop(columns=['SOS'], inplace=True)

    return srs

srs_m = simple_rating_system(regular_season_compact_results_m, mov_m)
srs_w = simple_rating_system(regular_season_compact_results_w, mov_w)

print(srs_m.head())
print(srs_w.head())

   Season  TeamID   avg_MOV  total_MOV  games_played  wins  losses   win_pct  \
0    1985    1102 -5.791667       -139            24     5      19  0.208333   
1    1985    1103 -3.043478        -70            23     9      14  0.391304   
2    1985    1104  7.800000        234            30    21       9  0.700000   
3    1985    1106 -3.791667        -91            24    10      14  0.416667   
4    1985    1108  7.960000        199            25    19       6  0.760000   

         SRS  
0  -5.875628  
1  -7.021700  
2  14.948632  
3 -10.042809  
4   2.207007  
   Season  TeamID    avg_MOV  total_MOV  games_played  wins  losses   win_pct  \
0    1998    3102 -20.625000       -495            24     4      20  0.166667   
1    1998    3103  -5.862069       -170            29    11      18  0.379310   
2    1998    3104  13.433333        403            30    21       9  0.700000   
3    1998    3106  -7.952381       -167            21     6      15  0.285714   
4    1998    3108   1.30

We will now estimate possessions in order to derive Offensive Rating, Defensive Rating, and Net Rating.

The formula for deriving possessions is Possessions = 0.96 \times (FGA + TO + 0.44 \times FTA - ORB)

In [27]:
def advanced_ratings(df: pd.DataFrame) -> pd.DataFrame:
    # Estimate possessions
    df['WPossessions'] = 0.96 * (df['WFGA'] + df['WTO'] + 0.44 * df['WFTA'] - df['WOR'])
    df['LPossessions'] = 0.96 * (df['LFGA'] + df['LTO'] + 0.44 * df['LFTA'] - df['LOR'])
    
    # Create dataframes for teams when they win
    w_teams = df[['Season', 'WTeamID', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WPossessions', 'LPossessions']].copy()
    w_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Create dataframes for teams when they lose
    l_teams = df[['Season', 'LTeamID', 'LScore', 'WScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LPossessions', 'WPossessions']].copy()
    l_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Combine both datasets
    all_games = pd.concat([w_teams, l_teams])
    
    # Group by Season and TeamID to get all metrics
    team_ratings = all_games.groupby(['Season', 'TeamID']).agg(
        total_points_scored=('PointsScored', 'sum'),
        total_points_allowed=('PointsAllowed', 'sum'),
        total_field_goals_made=('FGM', 'sum'),
        total_field_goals_attempted=('FGA', 'sum'),
        total_three_point_field_goals_made=('FGM3', 'sum'),
        total_three_point_field_goals_attempted=('FGA3', 'sum'),
        total_free_throws_made=('TFM', 'sum'),
        total_free_throws_attempted=('FTA', 'sum'),
        total_offensive_rebounds=('OR', 'sum'),
        total_defensive_rebounds=('DR', 'sum'),
        total_assists=('Ast', 'sum'),
        total_turnovers=('TO', 'sum'),
        total_steals=('Stl', 'sum'),
        total_blocks=('Blk', 'sum'),
        total_PFs=('PF', 'sum'),
        total_possessions=('OwnPossessions', 'sum'),
        total_opp_possessions=('OppPossessions', 'sum'),
        games_played=('PointsScored', 'count')
    ).reset_index()
    
    # Calculate advanced metrics
    team_ratings['total_points_diff'] = team_ratings['total_points_scored'] - team_ratings['total_points_allowed']
    team_ratings['points_per_game'] = team_ratings['total_points_scored'] / team_ratings['games_played']
    team_ratings['field_goal_percentage'] = 100 * team_ratings['total_field_goals_made'] / team_ratings['total_field_goals_attempted']
    team_ratings['field_goals_made_per_game'] = team_ratings['total_field_goals_made'] / team_ratings['games_played']
    team_ratings['three_point_percentage'] = 100 * team_ratings['total_three_point_field_goals_made'] / team_ratings['total_three_point_field_goals_attempted']
    team_ratings['three_point_field_goals_made_per_game'] = team_ratings['total_three_point_field_goals_made'] / team_ratings['games_played']
    team_ratings['free_throw_percentage'] = 100 * team_ratings['total_free_throws_made'] / team_ratings['total_free_throws_attempted']
    team_ratings['offensive_rebounds_per_game'] = team_ratings['total_offensive_rebounds'] / team_ratings['games_played']
    team_ratings['defensive_rebounds_per_game'] = team_ratings['total_defensive_rebounds'] / team_ratings['games_played']
    team_ratings['rebounds_per_game'] = team_ratings['offensive_rebounds_per_game'] + team_ratings['defensive_rebounds_per_game']
    team_ratings['assists_per_game'] = team_ratings['total_assists'] / team_ratings['games_played']
    team_ratings['turnovers_per_game'] = team_ratings['total_turnovers'] / team_ratings['games_played']
    team_ratings['steals_per_game'] = team_ratings['total_steals'] / team_ratings['games_played']
    team_ratings['blocks_per_game'] = team_ratings['total_blocks'] / team_ratings['games_played']
    team_ratings['personal_fouls_per_game'] = team_ratings['total_PFs'] / team_ratings['games_played']
    team_ratings['offensive_rating'] = 100 * team_ratings['total_points_scored'] / team_ratings['total_possessions']
    team_ratings['defensive_rating'] = 100 * team_ratings['total_points_allowed'] / team_ratings['total_opp_possessions']
    team_ratings['net_rating'] = team_ratings['offensive_rating'] - team_ratings['defensive_rating']
    
    return team_ratings

ratings_init_m = advanced_ratings(regular_season_detailed_results_m)
ratings_init_w = advanced_ratings(regular_season_detailed_results_w)

print(ratings_init_m.head())
print(ratings_init_w.head())

   Season  TeamID  total_points_scored  total_points_allowed  \
0    2003    1102                 1603                  1596   
1    2003    1103                 2127                  2110   
2    2003    1104                 1940                  1820   
3    2003    1105                 1866                  1993   
4    2003    1106                 1781                  1785   

   total_field_goals_made  total_field_goals_attempted  \
0                     536                         1114   
1                     733                         1508   
2                     673                         1601   
3                     634                         1602   
4                     656                         1548   

   total_three_point_field_goals_made  \
0                                 219   
1                                 147   
2                                 178   
3                                 197   
4                                 171   

   total_three_poin

At this point, we need to start narrowing down the variables of interest.
Here are the ones we will include:
- Season
- TeamID
- ConfAbbrev
- offensive_rating
- defensive_rating
- net_rating
- win_pct
- Seed
- Seed Difference
- rebounds_per_game
- assists_per_game
- steals_per_game
- blocks_per_game
- turnovers_per_game
- personal_fouls_per_game
- field_goal_percentage
- field_goals_made_per_game
- three_point_percentage
- three_point_field_goals_made_per_game
- total_points_scored

In [28]:
def get_team_conference(df: pd.DataFrame, conferences: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(conferences, on=['TeamID', 'Season'], how='left')
    return df

ratings_conference_m = get_team_conference(ratings_init_m, conferences_m)
ratings_conference_w = get_team_conference(ratings_init_w, conferences_w)

def get_seed(df: pd.DataFrame, seeding: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(seeding, on=['TeamID', 'Season'], how='left')
    return df

ratings_seed_m = get_seed(ratings_conference_m, seeding_by_season_m)
ratings_seed_w = get_seed(ratings_conference_w, seeding_by_season_w)

def drop_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns = ['total_points_allowed', 'total_field_goals_made', 'total_field_goals_attempted', 
               'total_three_point_field_goals_made', 'total_three_point_field_goals_attempted', 'total_free_throws_made', 
               'total_free_throws_attempted', 'total_offensive_rebounds', 'total_defensive_rebounds', 'total_assists', 
               'total_turnovers', 'total_steals', 'total_blocks', 'total_PFs', 'total_possessions', 
               'total_opp_possessions', 'games_played']
    df.drop(columns=columns, inplace=True)
    return df

# Drop unnecessary columns
ratings_m = drop_unnecessary_columns(ratings_seed_m)
ratings_w = drop_unnecessary_columns(ratings_seed_w)

print(ratings_m.head())
print(ratings_w.head())

   Season  TeamID  total_points_scored  total_points_diff  points_per_game  \
0    2003    1102                 1603                  7        57.250000   
1    2003    1103                 2127                 17        78.777778   
2    2003    1104                 1940                120        69.285714   
3    2003    1105                 1866               -127        71.769231   
4    2003    1106                 1781                 -4        63.607143   

   field_goal_percentage  field_goals_made_per_game  three_point_percentage  \
0              48.114901                  19.142857               37.564322   
1              48.607427                  27.148148               33.870968   
2              42.036227                  24.035714               32.014388   
3              39.575531                  24.384615               36.481481   
4              42.377261                  23.428571               34.615385   

   three_point_field_goals_made_per_game  free_throw_per

Now that we have the data, we can start to build the model using XGBoost.
The dataframes ratings_m and ratings_w should have all the relevant data we need to build the model.
But the model will need to be able to take in as input all possible matchups between two regular season teams in each season.

In [29]:
# Import relevant libraries

# xgboost related libraries:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

# sklearn related libraries:
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Setting up the data

y_actual_m = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\MNCAATourneyCompactResults.csv")
y_actual_w = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data\WNCAATourneyCompactResults.csv")

base_data_m = ratings_m.copy()
base_data_w = ratings_w.copy()




y_actual needs to be augmented to include all possible matchups. If a matchup is not in the tourney results, then WScore will be total points scored by the winning team and LScore will be total points scored by the losing team.
This is because the target variable is a float with range [0,1] that predicts the probability of the winning team winning the matchup.
We'll have to use point spread to determine the target variable. Calculations should be based on betting line conventions.
If two teams are in the tournamenet, then the target variable will be a sigmoid curve with a mean of 0.5 and a standard deviation of 0.1. 
The curve will be shifted to the right if the higher seed is the winning team and to the left if the lower seed is the winning team.

In [30]:
# We will need to reshape the data so that all possible matchups are included in the data. This will be done by creating a new dataframe that includes all possible matchups between two teams in each season.

def create_all_matchups(ratings_df):
    """
    Create all possible matchups between teams for each season,
    ensuring no duplicates (TeamA vs TeamB and TeamB vs TeamA)
    """
    # List to store all matchups
    all_matchups = []
    
    # For each season
    for season in ratings_df['Season'].unique():
        # Get teams for this season
        season_teams = ratings_df[ratings_df['Season'] == season]['TeamID'].tolist()
        
        # Create all possible matchups where TeamID1 < TeamID2 to avoid duplicates
        for i, team1 in enumerate(season_teams):
            for team2 in season_teams[i+1:]:
                all_matchups.append({'Season': season, 'TeamID1': team1, 'TeamID2': team2})
    
    # Convert to DataFrame
    matchups_df = pd.DataFrame(all_matchups)
    
    return matchups_df

# Create all possible matchups for men's and women's teams
matchups_m = create_all_matchups(ratings_m)
matchups_w = create_all_matchups(ratings_w)

print(f"Number of men's matchups: {len(matchups_m)}")
print(f"Number of women's matchups: {len(matchups_w)}")
print(matchups_m.head())


Number of men's matchups: 1381961
Number of women's matchups: 978221
   Season  TeamID1  TeamID2
0    2003     1102     1103
1    2003     1102     1104
2    2003     1102     1105
3    2003     1102     1106
4    2003     1102     1107


The next step is to fill in the target values for all possible combinations.
The formula involves the game's point spread (if available), the seed difference between 2 teams (if available), the difference in net rating between the 2 teams, and the difference in SRS between the two teams.
The evaluation method uses the Brier Score, which is used by the Kaggle competition to evaluate the model.

In [31]:
def fill_in_tourney_results(df: pd.DataFrame, tourney_results: pd.DataFrame) -> pd.DataFrame:
    """
    This function isn't the final model. It's just meant to fill in the target variable.
    """

    # Let's merge the matchups with the actual tournament results, we need to preserve all hypothetical matchups
    # First, standardize the tournament results to match our matchup format
    # Create a copy of tournament_results
    tourney_std = tourney_results.copy()
    
    # Ensure WTeamID and LTeamID are integers
    tourney_std['WTeamID'] = tourney_std['WTeamID'].astype(int)
    tourney_std['LTeamID'] = tourney_std['LTeamID'].astype(int)

    # Create columns to match our standardized format where TeamID1 < TeamID2
    tourney_std['TeamID1'] = tourney_std.apply(lambda row: min(row['WTeamID'], row['LTeamID']), axis=1)
    tourney_std['TeamID2'] = tourney_std.apply(lambda row: max(row['WTeamID'], row['LTeamID']), axis=1)
    
    # Add a column to track whether TeamID1 was the winner
    tourney_std['TeamID1_Won'] = tourney_std['TeamID1'] == tourney_std['WTeamID']

    # Add a data source column to identify the source of each result
    tourney_std['DataSource'] = 'Tournament'

    # Select only the columns we need from tourney_std
    tourney_cols = ['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore', 'DataSource']
    tourney_subset = tourney_std[tourney_cols]
    
    # Merge with our matchups dataframe
    result_df = df.merge(tourney_subset, on=['Season', 'TeamID1', 'TeamID2'], how='right')

    return result_df

y_actual_m = tournament_detailed_results_m
y_actual_w = tournament_detailed_results_w

merged_tourney_df_m = fill_in_tourney_results(matchups_m, y_actual_m)
merged_tourney_df_w = fill_in_tourney_results(matchups_w, y_actual_w)

display(merged_tourney_df_m)
display(merged_tourney_df_w)

,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2003,1411,1421,False,92,84,Tournament
1,2003,1112,1436,True,80,51,Tournament
2,2003,1113,1272,True,84,71,Tournament
3,2003,1141,1166,True,79,73,Tournament
4,2003,1143,1301,True,76,74,Tournament
...,...,...,...,...,...,...,...
1377,2024,1181,1301,False,76,64,Tournament
1378,2024,1345,1397,True,72,66,Tournament
1379,2024,1104,1163,False,86,72,Tournament
1380,2024,1301,1345,False,63,50,Tournament


,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2010,3124,3201,True,69,55,Tournament
1,2010,3173,3395,True,67,66,Tournament
2,2010,3181,3214,True,72,37,Tournament
3,2010,3199,3256,True,75,61,Tournament
4,2010,3207,3265,True,62,42,Tournament
...,...,...,...,...,...,...,...
889,2024,3163,3425,True,80,73,Tournament
890,2024,3234,3261,True,94,87,Tournament
891,2024,3163,3234,False,71,69,Tournament
892,2024,3301,3376,False,78,59,Tournament


In [32]:
def fill_in_regular_results(df: pd.DataFrame, regular_results: pd.DataFrame) -> pd.DataFrame:

    # We need to fill in the regular season matchups where the tournament results are missing
    regular_std = regular_results.copy()

    # Ensure WTeamID and LTeamID are integers
    regular_std['WTeamID'] = regular_std['WTeamID'].astype(int)
    regular_std['LTeamID'] = regular_std['LTeamID'].astype(int)

    # Create columns to match our standardized format where TeamID1 < TeamID2
    regular_std['TeamID1'] = regular_std.apply(lambda row: min(row['WTeamID'], row['LTeamID']), axis=1)
    regular_std['TeamID2'] = regular_std.apply(lambda row: max(row['WTeamID'], row['LTeamID']), axis=1)

    # Add a column to track whether TeamID1 was the winner, astype int to convert boolean to 0 or 1
    regular_std['TeamID1_Won'] = (regular_std['TeamID1'] == regular_std['WTeamID']).astype(int)
    
    # Add a column to identify this as regular season data
    regular_std['DataSource'] = 'RegularSeason'
    
    # Select only the columns we need from regular_std
    regular_cols = ['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore', 'DataSource']
    regular_subset = regular_std[regular_cols]
        
    # Merge with dataframe but only keep rows that exist in regular_subset
    result_df = df.merge(regular_subset, on=['Season', 'TeamID1', 'TeamID2'], how='right')

    return result_df

    # # Fill in the target variable
    # # The target variable is a function of the game's point spread, the seed difference, and the difference in net rating between the two teams
    # # The point spread is the difference in points scored between the winning and losing teams, the bigger the point spread, the more likely the winning team is to win
    # # The seed difference is the difference in seed between the two teams, the bigger the seed difference, the more likely the higher seed is to win
    # # The difference in net rating is the difference in net rating between the two teams, the bigger the difference, the more likely the team with the higher net rating is to win
    # # Target has a range of [0,1], thus a sigmoid function is used to normalize the target variable
    # df['target'] = 1 / (1 + np.exp(-0.1 * (df['MOV'] - 0.1 * df['seed_diff'] - 0.1 * df['net_rating_diff'] - 0.1 * df['srs_diff'])))

    # return df

merged_regular_df_m = fill_in_regular_results(matchups_m, regular_season_detailed_results_m)

display(merged_regular_df_m)

merged_regular_df_w = fill_in_regular_results(matchups_w, regular_season_detailed_results_w)

display(merged_regular_df_w)

,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2003,1104,1328,1,68,62,RegularSeason
1,2003,1272,1393,1,70,63,RegularSeason
2,2003,1266,1437,1,73,61,RegularSeason
3,2003,1296,1457,1,56,50,RegularSeason
4,2003,1208,1400,0,77,71,RegularSeason
...,...,...,...,...,...,...,...
118444,2025,1182,1433,0,71,62,RegularSeason
118445,2025,1107,1436,0,79,71,RegularSeason
118446,2025,1199,1438,0,60,57,RegularSeason
118447,2025,1428,1452,0,71,69,RegularSeason


,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2010,3103,3237,1,63,49,RegularSeason
1,2010,3104,3399,1,73,68,RegularSeason
2,2010,3110,3224,1,71,59,RegularSeason
3,2010,3111,3267,1,63,58,RegularSeason
4,2010,3119,3447,1,74,70,RegularSeason
...,...,...,...,...,...,...,...
81303,2025,3150,3396,0,60,54,RegularSeason
81304,2025,3409,3455,1,58,46,RegularSeason
81305,2025,3408,3412,0,66,64,RegularSeason
81306,2025,3187,3427,0,67,48,RegularSeason


In [33]:
def fill_in_other_results(df: pd.DataFrame, tourney_results: pd.DataFrame, regular_results: pd.DataFrame, ratings: pd.DataFrame, srs: pd.DataFrame, alpha=0.1) -> pd.DataFrame:
    # Filter our the rows in df that already exist in tourney_results and regular_results
    # They key is TeamID1, TeamID2, and Season

    tourney_results_rows = tourney_results[['Season', 'TeamID1', 'TeamID2']].copy()
    regular_results_rows = regular_results[['Season', 'TeamID1', 'TeamID2']].copy()

    existing_results = pd.concat([tourney_results_rows, regular_results_rows])

    # Filter out existing_results from df
    # Merge with indicator to identify rows that exist only in df

    df = df.merge(existing_results, on=['Season', 'TeamID1', 'TeamID2'], how='left', indicator=True)

    # Keep only rows that are in df but not in existing_results
    df = df[df['_merge'] == 'left_only'].drop(columns=['_merge'])

    # For the remaining rows, create columns TeamID1_Won, WScore, LScore, DataSource
    # WScore is the points per game for the winning team for the season
    # LScore is the points per game for the losing team for the season
    # DataSource is 'Other'
    # TeamID1_Won is 1 if TeamID1 has a higher points per game than TeamID2, 0 otherwise

    # Merge with ratings to get points per game for each team
    ratings_subset = ratings[['Season', 'TeamID', 'points_per_game']]

    df = df.merge(ratings_subset, left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'], how='left')

    df.rename(columns={'points_per_game': 'Team1_PPG'}, inplace=True)

    df = df.merge(ratings_subset, left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'], how='left')

    df.rename(columns={'points_per_game': 'Team2_PPG'}, inplace=True)

    # We need to adjust the Team1_PPG and Team2_PPG columns by SRS, which is a measure of team strength
    # The higher the SRS, the stronger the team
    # We will adjust the points per game by the SRS

    srs_subset = srs[['Season', 'TeamID', 'SRS']]
    df = df.merge(srs_subset, left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'], how='left')

    drop_columns = ['TeamID_x', 'TeamID_y', 'TeamID']

    df.drop(columns=drop_columns, inplace=True)

    df = df.merge(srs_subset, left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'], how='left')

    drop_columns = ['TeamID']

    df.drop(columns=drop_columns, inplace=True)

    df.rename(columns={'SRS_x': 'Team1_SRS', 'SRS_y': 'Team2_SRS'}, inplace=True)

    # First, calculate the difference in SRS betweeen the two teams, with a greater difference indicating a stronger team
    df['SRS_diff'] = df['Team1_SRS'] - df['Team2_SRS']
    df['SRS_scaled'] = 2 / (1 + np.exp(-alpha * df['SRS_diff']))
    df['Expected_PPG1'] = df['Team1_PPG'] * df['SRS_scaled']
    df['Expected_PPG2'] = df['Team2_PPG'] * (2 - df['SRS_scaled'])


    # Create TeamID_Won, WScore, LScore, and DataSource columns
    df['TeamID1_Won'] = (df['Expected_PPG1'] > df['Expected_PPG2']).astype(int)
    df['WScore'] = df['Expected_PPG1']
    df['LScore'] = df['Expected_PPG2']
    df['DataSource'] = 'Other'

    # Round the scores to integers
    df['WScore'] = df['WScore'].round().astype(int)
    df['LScore'] = df['LScore'].round().astype(int)

    print(df.columns)

    # Drop unnecessary columns
    df.drop(columns=['Team1_PPG', 'Team2_PPG', 'Team1_SRS', 'Team2_SRS', 'SRS_diff', 'SRS_scaled','Expected_PPG1', 'Expected_PPG2'], inplace=True)

    return df
    
merged_other_df_m = fill_in_other_results(matchups_m, merged_tourney_df_m, merged_regular_df_m, base_data_m, srs_m, alpha=0.006)
display(merged_other_df_m)

# # Show rows where (TeamID1 is in the bottom 10% of SRS and TeamID2 is in the top 10% of SRS) or (TeamID1 is in the top 10% of SRS and TeamID2 is in the bottom 10% of SRS)
# top_bottom_matchups_m = merged_other_df_m[
#     ((merged_other_df_m['Team1_SRS'] < merged_other_df_m['Team1_SRS'].quantile(0.1)) & 
#      (merged_other_df_m['Team2_SRS'] > merged_other_df_m['Team2_SRS'].quantile(0.9))) | 
#     ((merged_other_df_m['Team1_SRS'] > merged_other_df_m['Team1_SRS'].quantile(0.9)) & 
#      (merged_other_df_m['Team2_SRS'] < merged_other_df_m['Team2_SRS'].quantile(0.1)))
# ]

# display(top_bottom_matchups_m)

# # Find the average point spread (absolute value) for these matchups, which is the difference in points per game between the two teams
# avg_point_spread_m = top_bottom_matchups_m['WScore'].sub(top_bottom_matchups_m['LScore']).abs().mean()
# print(f"Average point spread for top 10% vs bottom 10% matchups: {avg_point_spread_m:.2f}")

# Average point spread for top 10% vs bottom 10% matchups: 26.34

merged_other_df_w = fill_in_other_results(matchups_w, merged_tourney_df_w, merged_regular_df_w, base_data_w, srs_w, alpha=0.007)
display(merged_other_df_w)

# # Show rows where (TeamID1 is in the bottom 10% of SRS and TeamID2 is in the top 10% of SRS) or (TeamID1 is in the top 10% of SRS and TeamID2 is in the bottom 10% of SRS)
# top_bottom_matchups_w = merged_other_df_w[
#     ((merged_other_df_w['Team1_SRS'] < merged_other_df_w['Team1_SRS'].quantile(0.1)) & 
#      (merged_other_df_w['Team2_SRS'] > merged_other_df_w['Team2_SRS'].quantile(0.9))) | 
#     ((merged_other_df_w['Team1_SRS'] > merged_other_df_w['Team1_SRS'].quantile(0.9)) & 
#      (merged_other_df_w['Team2_SRS'] < merged_other_df_w['Team2_SRS'].quantile(0.1)))
# ]

# display(top_bottom_matchups_m)

# # Find the average point spread (absolute value) for these matchups, which is the difference in points per game between the two teams
# avg_point_spread_w = top_bottom_matchups_w['WScore'].sub(top_bottom_matchups_w['LScore']).abs().mean()
# print(f"Average point spread for top 10% vs bottom 10% matchups: {avg_point_spread_w:.2f}")

# Average point spread for top 10% vs bottom 10% matchups: 40.19

Index(['Season', 'TeamID1', 'TeamID2', 'Team1_PPG', 'Team2_PPG', 'Team1_SRS',
       'Team2_SRS', 'SRS_diff', 'SRS_scaled', 'Expected_PPG1', 'Expected_PPG2',
       'TeamID1_Won', 'WScore', 'LScore', 'DataSource'],
      dtype='object')


,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2003,1102,1103,0,58,78,Other
1,2003,1102,1104,0,55,72,Other
2,2003,1102,1105,0,60,68,Other
3,2003,1102,1106,0,59,61,Other
4,2003,1102,1107,0,60,63,Other
...,...,...,...,...,...,...,...
1296997,2025,1477,1478,0,65,71,Other
1296998,2025,1477,1479,0,65,65,Other
1296999,2025,1477,1480,0,65,67,Other
1297000,2025,1478,1480,1,72,68,Other


Index(['Season', 'TeamID1', 'TeamID2', 'Team1_PPG', 'Team2_PPG', 'Team1_SRS',
       'Team2_SRS', 'SRS_diff', 'SRS_scaled', 'Expected_PPG1', 'Expected_PPG2',
       'TeamID1_Won', 'WScore', 'LScore', 'DataSource'],
      dtype='object')


,Season,TeamID1,TeamID2,TeamID1_Won,WScore,LScore,DataSource
0,2010,3102,3103,0,49,66,Other
1,2010,3102,3104,0,48,69,Other
2,2010,3102,3105,0,52,61,Other
3,2010,3102,3106,0,51,56,Other
4,2010,3102,3107,0,50,58,Other
...,...,...,...,...,...,...,...
919336,2025,3477,3478,1,64,52,Other
919337,2025,3477,3479,1,63,61,Other
919338,2025,3477,3480,0,61,67,Other
919339,2025,3478,3480,0,52,69,Other


In [ ]:
def merge_all_results(tourney_df: pd.DataFrame, regular_df: pd.DataFrame, other_df: pd.DataFrame, ratings: pd.DataFrame, srs: pd.DataFrame) -> pd.DataFrame:
    # Note that if a Season, TeamID1, TeamID2 combination exists in multiple dataframes, the order of precedence is tourney_df, regular_df, other_df
    # This means that if a row exists in tourney_df, it will be kept, and the same for regular_df and other_df

    merged_df = tourney_df.merge(regular_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('_tourney', '_regular'))
    
    # Handle precedence - tournament results take priority
    # First, create columns without suffixes using tournament data when available, regular data otherwise
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        merged_df[col] = merged_df[f'{col}_tourney'].combine_first(merged_df[f'{col}_regular'])
    
    # Drop the columns with suffixes as we've consolidated them
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_tourney') or col.endswith('_regular')]
    merged_df = merged_df.drop(columns=cols_to_drop)
    
    # Now merge with other results, again with other having lowest precedence
    merged_df = merged_df.merge(other_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('', '_other'))
    
    # Again handle precedence - merged results take priority over other
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        if f'{col}_other' in merged_df.columns:
            merged_df[col] = merged_df[col].combine_first(merged_df[f'{col}_other'])
    
    # Drop the columns with _other suffix
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_other')]
    merged_df = merged_df.drop(columns=cols_to_drop)

    # display(merged_df)

    # Merge with ratings and SRS data for team statistics
    ratings_team1 = ratings.copy()
    for col in ratings_team1.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    # Merge with explicitly renamed columns to avoid suffix issues
    merged_df = merged_df.merge(ratings_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    # display(merged_df)
    
    ratings_team2 = ratings.copy()
    for col in ratings_team2.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(ratings_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left')

    # display(merged_df)
    
    srs_team1 = srs.copy()
    for col in srs_team1.columns:
        if col not in ['Season', 'TeamID']:
            srs_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    # display(merged_df)
    
    srs_team2 = srs.copy()
    for col in srs_team2.columns:
        if col not in ['Season', 'TeamID']:
            srs_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left', suffixes=('', '_srs2'))

    # display(merged_df)

    # Scan for duplicate Season, TeamID1, TeamID2, and drop duplicates per the tourney, regular, other precedence
    merged_df = merged_df.drop_duplicates(subset=['Season', 'TeamID1', 'TeamID2'], keep='first')

    print(merged_df.columns)
    
    merged_df['Seed_team1'].fillna('W16', inplace=True)
    merged_df['Seed_team2'].fillna('W16', inplace=True)

    dropped_columns = ['TeamID_x', 'total_points_scored_team1', 'TeamID_y', 'total_points_scored_team2']

    merged_df.drop(columns=dropped_columns, inplace=True)


    # Target variable will be 1 if TeamID1 wins, 0 otherwise
    merged_df.rename(columns={'TeamID1_Won': 'target'}, inplace=True)

    # Move target to the last column
    cols = list(merged_df.columns)
    cols.remove('target')
    merged_df = merged_df[cols + ['target']]

    return merged_df

   
combined_df_m = merge_all_results(merged_tourney_df_m, merged_regular_df_m, merged_other_df_m, ratings_m, srs_m)
display(combined_df_m)

Index(['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore',
       'DataSource', 'TeamID_x', 'total_points_scored_team1',
       'total_points_diff_team1', 'points_per_game_team1',
       'field_goal_percentage_team1', 'field_goals_made_per_game_team1',
       'three_point_percentage_team1',
       'three_point_field_goals_made_per_game_team1',
       'free_throw_percentage_team1', 'offensive_rebounds_per_game_team1',
       'defensive_rebounds_per_game_team1', 'rebounds_per_game_team1',
       'assists_per_game_team1', 'turnovers_per_game_team1',
       'steals_per_game_team1', 'blocks_per_game_team1',
       'personal_fouls_per_game_team1', 'offensive_rating_team1',
       'defensive_rating_team1', 'net_rating_team1', 'ConfAbbrev_team1',
       'Seed_team1', 'TeamID_y', 'total_points_scored_team2',
       'total_points_diff_team2', 'points_per_game_team2',
       'field_goal_percentage_team2', 'field_goals_made_per_game_team2',
       'three_point_percentage_team2',
   

C:\Users\henry-cao-local\AppData\Local\Temp\ipykernel_27952\735779809.py:77: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Seed_team1'].fillna('W16', inplace=True)
C:\Users\henry-cao-local\AppData\Local\Temp\ipykernel_27952\735779809.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,SRS_team1,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target
0,2003,1102,1103,58.0,78.0,Other,7,57.250000,48.114901,19.142857,...,2.266120,1103,0.629630,17,27,13,14,0.481481,-0.416138,0.0
1,2003,1102,1104,55.0,72.0,Other,7,57.250000,48.114901,19.142857,...,2.266120,1104,4.285714,120,28,17,11,0.607143,14.222003,0.0
2,2003,1102,1105,60.0,68.0,Other,7,57.250000,48.114901,19.142857,...,2.266120,1105,-4.884615,-127,26,7,19,0.269231,-16.032281,0.0
3,2003,1102,1106,59.0,61.0,Other,7,57.250000,48.114901,19.142857,...,2.266120,1106,-0.142857,-4,28,13,15,0.464286,-9.467136,0.0
4,2003,1102,1107,60.0,63.0,Other,7,57.250000,48.114901,19.142857,...,2.266120,1107,-9.785714,-274,28,7,21,0.250000,-14.606844,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416767,2025,1477,1479,65.0,65.0,Other,-326,64.354839,41.598600,23.000000,...,-12.234262,1479,-5.964286,-167,28,12,16,0.428571,-15.068266,0.0
1416768,2025,1477,1480,65.0,67.0,Other,-326,64.354839,41.598600,23.000000,...,-12.234262,1480,-11.066667,-332,30,5,25,0.166667,-14.876635,0.0
1416769,2025,1478,1479,79.0,63.0,RegularSeason,-265,72.103448,44.623990,24.758621,...,-17.121246,1479,-5.964286,-167,28,12,16,0.428571,-15.068266,1.0
1416771,2025,1478,1480,72.0,68.0,Other,-265,72.103448,44.623990,24.758621,...,-17.121246,1480,-11.066667,-332,30,5,25,0.166667,-14.876635,1.0


[0.0 1.0]


In [38]:
combined_df_w = merge_all_results(merged_tourney_df_w, merged_regular_df_w, merged_other_df_w, ratings_w, srs_w)
display(combined_df_w)

Index(['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore',
       'DataSource', 'TeamID_x', 'total_points_scored_team1',
       'total_points_diff_team1', 'points_per_game_team1',
       'field_goal_percentage_team1', 'field_goals_made_per_game_team1',
       'three_point_percentage_team1',
       'three_point_field_goals_made_per_game_team1',
       'free_throw_percentage_team1', 'offensive_rebounds_per_game_team1',
       'defensive_rebounds_per_game_team1', 'rebounds_per_game_team1',
       'assists_per_game_team1', 'turnovers_per_game_team1',
       'steals_per_game_team1', 'blocks_per_game_team1',
       'personal_fouls_per_game_team1', 'offensive_rating_team1',
       'defensive_rating_team1', 'net_rating_team1', 'ConfAbbrev_team1',
       'Seed_team1', 'TeamID_y', 'total_points_scored_team2',
       'total_points_diff_team2', 'points_per_game_team2',
       'field_goal_percentage_team2', 'field_goals_made_per_game_team2',
       'three_point_percentage_team2',
   

C:\Users\henry-cao-local\AppData\Local\Temp\ipykernel_27952\1342603188.py:77: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Seed_team1'].fillna('W16', inplace=True)
C:\Users\henry-cao-local\AppData\Local\Temp\ipykernel_27952\1342603188.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,SRS_team1,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target
0,2010,3102,3103,49.0,66.0,Other,-559,51.821429,36.021505,19.142857,...,-16.747439,3103,2.600000,78,30,17,13,0.566667,0.955104,0.0
1,2010,3102,3104,48.0,69.0,Other,-559,51.821429,36.021505,19.142857,...,-16.747439,3104,-2.600000,-78,30,12,18,0.400000,2.849553,0.0
2,2010,3102,3105,52.0,61.0,Other,-559,51.821429,36.021505,19.142857,...,-16.747439,3105,-3.962963,-107,27,14,13,0.518519,-15.735173,0.0
3,2010,3102,3106,51.0,56.0,Other,-559,51.821429,36.021505,19.142857,...,-16.747439,3106,-2.275862,-66,29,12,17,0.413793,-14.673963,0.0
4,2010,3102,3107,50.0,58.0,Other,-559,51.821429,36.021505,19.142857,...,-16.747439,3107,-5.400000,-162,30,11,19,0.366667,-8.578891,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001469,2025,3477,3479,63.0,61.0,Other,-285,61.076923,37.628205,22.576923,...,-14.834255,3479,-7.391304,-170,23,6,17,0.260870,-21.676320,1.0
1001470,2025,3477,3480,61.0,67.0,Other,-285,61.076923,37.628205,22.576923,...,-14.834255,3480,-5.074074,-137,27,11,16,0.407407,-12.562717,0.0
1001471,2025,3478,3479,63.0,52.0,RegularSeason,-444,54.250000,35.163308,18.071429,...,-26.831148,3479,-7.391304,-170,23,6,17,0.260870,-21.676320,0.0
1001473,2025,3478,3480,52.0,69.0,Other,-444,54.250000,35.163308,18.071429,...,-26.831148,3480,-5.074074,-137,27,11,16,0.407407,-12.562717,0.0


Now we do some advanced feature engineering that incorporates betting odds and seed probabilities.

In [20]:
mensProbabilities_df = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_fivethirtyeight\mensProbabilitiesTable2024.csv", index_col = 'player') #source: https://github.com/gotoConversion/goto_conversion
mensProbabilities_df = mensProbabilities_df.drop('Elo_Rating', axis=1)

womensProbabilities_df = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_fivethirtyeight\womensProbabilitiesTable2024.csv", index_col = 'player') #source: https://github.com/gotoConversion/goto_conversion
womensProbabilities_df = womensProbabilities_df.drop('Elo_Rating', axis=1)

In [22]:
year = 2025
mensTeamSeeds_df = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\MNCAATourneySeeds.csv")
mensTeamSeeds2025_df = mensTeamSeeds_df.iloc[[x == year for x in mensTeamSeeds_df['Season']]]
womensTeamSeeds_df = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_updated\WNCAATourneySeeds.csv")
womensTeamSeeds2025_df = womensTeamSeeds_df.iloc[[x == year for x in womensTeamSeeds_df['Season']]]

In [ ]:
# Using the seed, get the round of the match
def get_roundOfMatch(df: pd.DataFrame) -> pd.DataFrame:

    slotMap = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

    team1_seed = df['seed_team1']
    team2_seed = df['seed_team2']

    isFirstFourMatch = team1_seed[:3] == team2_seed[:3]
    if isFirstFourMatch:
        return 1
    
    team1_region = str(team1_seed[:1])
    team2_region = str(team2_seed[:1])

    team1_seedNumber = int(team1_seed[1:3])
    team2_seedNumber = int(team2_seed[1:3])

    isRegionSame = team1_region == team2_region
    if not isRegionSame:

        isTeam1_regionWX = team1_region in ['W', 'X']
        isTeam2_regionWX = team2_region in ['W', 'X']

        if isTeam1_regionWX and isTeam2_regionWX:
            return 6
        elif (not isTeam1_regionWX) and (not isTeam2_regionWX):
            return 6
        else:
            return 7
    
    else:

        team1_slot = slotMap.index(team1_seedNumber)
        team2_slot = slotMap.index(team2_seedNumber)

        isRound2 = (team1_slot // 2) == (team2_slot // 2)
        if isRound2:
            return 2
        
        isRound3 = (team1_slot // 4) == (team2_slot // 4)
        if isRound3:
            return 3

        isRound4 = (team1_slot // 8) == (team2_slot // 8)
        if isRound4:
            return 4
        
        else:
            return 5
        
def get_flag_list(submission_df, mens_seeds_df, womens_seeds_df):
    """
    Determine the tournament round for each matchup in the submission dataframe
    
    Args:
        submission_df: DataFrame with matchup IDs
        mens_seeds_df: DataFrame with men's team seeds
        womens_seeds_df: DataFrame with women's team seeds
        
    Returns:
        List of round numbers for each matchup
    """
    flag_list = []
    
    for i in range(submission_df.shape[0]):
        submission_row = submission_df.iloc[i,0].split('_')
        season = int(submission_row[0])
        team1 = int(submission_row[1])
        team2 = int(submission_row[2])
        
        # Determine if men's or women's game
        is_mens_game = (1000 < team1 < 2000) and (1000 < team2 < 2000)
        
        if is_mens_game:
            seeds_df = mens_seeds_df[mens_seeds_df['Season'] == season]
        else:
            seeds_df = womens_seeds_df[womens_seeds_df['Season'] == season]
        
        # Get the seeds for each team
        team1_seed_row = seeds_df[seeds_df['TeamID'] == team1]
        team2_seed_row = seeds_df[seeds_df['TeamID'] == team2]
        
        if team1_seed_row.empty or team2_seed_row.empty:
            # If we can't find the seeds, we can't determine the round
            flag_list.append(0)
            continue
        
        team1_seed = team1_seed_row['Seed'].values[0]
        team2_seed = team2_seed_row['Seed'].values[0]
        
        # Create a dictionary to pass to get_roundOfMatch
        match_dict = {'seed_team1': team1_seed, 'seed_team2': team2_seed}
        
        # Get the round of the match
        round_num = get_roundOfMatch(match_dict)
        flag_list.append(round_num)
    
    return flag_list

def set_optimalStrategy(combined_df, submission_df, riskTeam, riskTeamToWinRound):
    """
    Add an optimal probability column to combined_df based on the risk strategy
    
    Args:
        combined_df: DataFrame with match information
        submission_df: DataFrame with tournament predictions
        riskTeam: TeamID to optimize for
        riskTeamToWinRound: How far the risk team is expected to advance
        
    Returns:
        combined_df with added optimal_probability column
    """
    # Initialize optimal_probability column with current target values
    combined_df['optimal_probability'] = combined_df['target']
    
    # Get round information for submission matchups
    flag_list = get_flag_list(submission_df, mensTeamSeeds2025_df, womensTeamSeeds2025_df)
    
    # Create mapping from ID to round
    round_mapping = {}
    for i, row_id in enumerate(submission_df['ID']):
        round_mapping[row_id] = flag_list[i]
    
    # Filter for tournament games
    tournament_games = combined_df[combined_df['DataSource'] == 'Tournament']
    
    # For each tournament game, set optimal probability based on the risk strategy
    for idx, row in tournament_games.iterrows():
        season = row['Season']
        team1 = row['TeamID1']
        team2 = row['TeamID2']
        
        # Create ID in the same format as submission_df
        match_id = f"{season}_{team1}_{team2}"
        
        # Get the round if it exists in the mapping
        match_round = round_mapping.get(match_id, 0)
        
        if match_round > 0:
            # If team1 is the risk team and the round is within our target
            if team1 == riskTeam and match_round <= riskTeamToWinRound:
                combined_df.at[idx, 'optimal_probability'] = 1.0
            
            # If team2 is the risk team and the round is within our target
            elif team2 == riskTeam and match_round <= riskTeamToWinRound:
                combined_df.at[idx, 'optimal_probability'] = 0.0
    
    return combined_df

riskTeam = 1179
riskTeamToWinRound = 2
submission_df = pd.read_csv(r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\data_fivethirtyeight\submission.csv")  

augmented_df_m = set_optimalStrategy(combined_df_m, submission_df, riskTeam, riskTeamToWinRound)
display(augmented_df_m)

augmented_df_w = set_optimalStrategy(combined_df_w, submission_df, riskTeam, riskTeamToWinRound)
display(augmented_df_w)

,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target,optimal_probability
0,2003,1102,1103,58.0,78.0,Other,7,57.250000,48.114901,19.142857,...,1103,0.629630,17,27,13,14,0.481481,-0.416138,0.0,0.0
1,2003,1102,1104,55.0,72.0,Other,7,57.250000,48.114901,19.142857,...,1104,4.285714,120,28,17,11,0.607143,14.222003,0.0,0.0
2,2003,1102,1105,60.0,68.0,Other,7,57.250000,48.114901,19.142857,...,1105,-4.884615,-127,26,7,19,0.269231,-16.032281,0.0,0.0
3,2003,1102,1106,59.0,61.0,Other,7,57.250000,48.114901,19.142857,...,1106,-0.142857,-4,28,13,15,0.464286,-9.467136,0.0,0.0
4,2003,1102,1107,60.0,63.0,Other,7,57.250000,48.114901,19.142857,...,1107,-9.785714,-274,28,7,21,0.250000,-14.606844,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416767,2025,1477,1479,65.0,65.0,Other,-326,64.354839,41.598600,23.000000,...,1479,-5.964286,-167,28,12,16,0.428571,-15.068266,0.0,0.0
1416768,2025,1477,1480,65.0,67.0,Other,-326,64.354839,41.598600,23.000000,...,1480,-11.066667,-332,30,5,25,0.166667,-14.876635,0.0,0.0
1416769,2025,1478,1479,79.0,63.0,RegularSeason,-265,72.103448,44.623990,24.758621,...,1479,-5.964286,-167,28,12,16,0.428571,-15.068266,1.0,1.0
1416771,2025,1478,1480,72.0,68.0,Other,-265,72.103448,44.623990,24.758621,...,1480,-11.066667,-332,30,5,25,0.166667,-14.876635,1.0,1.0


,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target,optimal_probability
0,2010,3102,3103,49.0,66.0,Other,-559,51.821429,36.021505,19.142857,...,3103,2.600000,78,30,17,13,0.566667,0.955104,0.0,0.0
1,2010,3102,3104,48.0,69.0,Other,-559,51.821429,36.021505,19.142857,...,3104,-2.600000,-78,30,12,18,0.400000,2.849553,0.0,0.0
2,2010,3102,3105,52.0,61.0,Other,-559,51.821429,36.021505,19.142857,...,3105,-3.962963,-107,27,14,13,0.518519,-15.735173,0.0,0.0
3,2010,3102,3106,51.0,56.0,Other,-559,51.821429,36.021505,19.142857,...,3106,-2.275862,-66,29,12,17,0.413793,-14.673963,0.0,0.0
4,2010,3102,3107,50.0,58.0,Other,-559,51.821429,36.021505,19.142857,...,3107,-5.400000,-162,30,11,19,0.366667,-8.578891,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001469,2025,3477,3479,63.0,61.0,Other,-285,61.076923,37.628205,22.576923,...,3479,-7.391304,-170,23,6,17,0.260870,-21.676320,1.0,1.0
1001470,2025,3477,3480,61.0,67.0,Other,-285,61.076923,37.628205,22.576923,...,3480,-5.074074,-137,27,11,16,0.407407,-12.562717,0.0,0.0
1001471,2025,3478,3479,63.0,52.0,RegularSeason,-444,54.250000,35.163308,18.071429,...,3479,-7.391304,-170,23,6,17,0.260870,-21.676320,0.0,0.0
1001473,2025,3478,3480,52.0,69.0,Other,-444,54.250000,35.163308,18.071429,...,3480,-5.074074,-137,27,11,16,0.407407,-12.562717,0.0,0.0


In [43]:
# Print unique values of target variable
print(augmented_df_m['target'].unique())

# Display only Tourney teams in augmented_df
tourney_teams_m = augmented_df_m[augmented_df_m['DataSource'] == 'Tournament']
display(tourney_teams_m)

combined_tourney_teams_m = combined_df_m[combined_df_m['DataSource'] == 'Tournament']
display(combined_tourney_teams_m)

tourney_teams_w = augmented_df_w[augmented_df_w['DataSource'] == 'Tournament']
display(tourney_teams_w)

combined_tourney_teams_w = combined_df_w[combined_df_w['DataSource'] == 'Tournament']
display(combined_tourney_teams_w)

[0.0 1.0]


,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target,optimal_probability
782,2003,1104,1231,67.0,62.0,Tournament,120,69.285714,42.036227,24.035714,...,1231,3.531250,113,32,20,12,0.625000,13.194010,False,False
3069,2003,1112,1211,96.0,95.0,Tournament,419,85.214286,46.141304,30.321429,...,1211,8.870968,275,31,23,8,0.741935,12.146328,True,True
3096,2003,1112,1242,78.0,75.0,Tournament,419,85.214286,46.141304,30.321429,...,1242,14.966667,449,30,23,7,0.766667,24.846002,False,False
3165,2003,1112,1323,88.0,71.0,Tournament,419,85.214286,46.141304,30.321429,...,1323,8.903226,276,31,22,9,0.709677,17.787715,True,True
3275,2003,1112,1436,80.0,51.0,Tournament,419,85.214286,46.141304,30.321429,...,1436,4.655172,135,29,19,10,0.655172,-0.868644,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343027,2024,1361,1412,69.0,65.0,Tournament,219,74.187500,43.903743,25.656250,...,1412,1.575758,52,33,22,11,0.666667,5.196931,True,True
1343079,2024,1361,1463,85.0,57.0,Tournament,219,74.187500,43.903743,25.656250,...,1463,5.413793,157,29,20,9,0.689655,4.840647,True,True
1345655,2024,1389,1397,83.0,49.0,Tournament,58,65.096774,39.755530,22.032258,...,1397,11.593750,371,32,24,8,0.750000,21.778119,False,False
1346194,2024,1395,1429,88.0,72.0,Tournament,264,79.242424,46.488625,28.484848,...,1429,7.419355,230,31,25,6,0.806452,12.156898,False,False


,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,SRS_team1,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target
782,2003,1104,1231,67.0,62.0,Tournament,120,69.285714,42.036227,24.035714,...,14.222003,1231,3.531250,113,32,20,12,0.625000,13.194010,False
3069,2003,1112,1211,96.0,95.0,Tournament,419,85.214286,46.141304,30.321429,...,23.846575,1211,8.870968,275,31,23,8,0.741935,12.146328,True
3096,2003,1112,1242,78.0,75.0,Tournament,419,85.214286,46.141304,30.321429,...,23.846575,1242,14.966667,449,30,23,7,0.766667,24.846002,False
3165,2003,1112,1323,88.0,71.0,Tournament,419,85.214286,46.141304,30.321429,...,23.846575,1323,8.903226,276,31,22,9,0.709677,17.787715,True
3275,2003,1112,1436,80.0,51.0,Tournament,419,85.214286,46.141304,30.321429,...,23.846575,1436,4.655172,135,29,19,10,0.655172,-0.868644,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343027,2024,1361,1412,69.0,65.0,Tournament,219,74.187500,43.903743,25.656250,...,15.302059,1412,1.575758,52,33,22,11,0.666667,5.196931,True
1343079,2024,1361,1463,85.0,57.0,Tournament,219,74.187500,43.903743,25.656250,...,15.302059,1463,5.413793,157,29,20,9,0.689655,4.840647,True
1345655,2024,1389,1397,83.0,49.0,Tournament,58,65.096774,39.755530,22.032258,...,-4.076225,1397,11.593750,371,32,24,8,0.750000,21.778119,False
1346194,2024,1395,1429,88.0,72.0,Tournament,264,79.242424,46.488625,28.484848,...,15.655324,1429,7.419355,230,31,25,6,0.806452,12.156898,False


,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target,optimal_probability
3898,2010,3114,3210,63.0,53.0,Tournament,462,66.843750,44.854155,25.468750,...,3210,8.562500,274,32,23,9,0.718750,16.061312,True,True
4011,2010,3114,3328,60.0,44.0,Tournament,462,66.843750,44.854155,25.468750,...,3328,5.030303,166,33,23,10,0.696970,22.591927,False,False
6081,2010,3122,3397,75.0,42.0,Tournament,-179,66.187500,38.721606,22.906250,...,3397,17.406250,557,32,30,2,0.937500,33.372460,False,False
6516,2010,3124,3163,70.0,50.0,Tournament,488,72.093750,46.311703,26.093750,...,3163,34.757576,1147,33,33,0,1.000000,49.614213,False,False
6534,2010,3124,3181,51.0,48.0,Tournament,488,72.093750,46.311703,26.093750,...,3181,16.531250,529,32,27,5,0.843750,28.913093,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926515,2024,3343,3452,63.0,53.0,Tournament,399,69.724138,45.838084,27.724138,...,3452,16.870968,523,31,24,7,0.774194,28.204894,False,False
928001,2024,3355,3428,68.0,54.0,Tournament,408,73.766667,48.232475,26.833333,...,3428,14.000000,420,30,20,10,0.666667,30.061336,False,False
931729,2024,3394,3425,87.0,55.0,Tournament,152,65.185185,40.090674,22.925926,...,3425,12.516129,388,31,26,5,0.838710,30.939475,False,False
931998,2024,3397,3453,92.0,63.0,Tournament,195,75.935484,43.015633,27.516129,...,3453,16.406250,525,32,26,6,0.812500,15.052416,True,True


,Season,TeamID1,TeamID2,WScore,LScore,DataSource,total_points_diff_team1,points_per_game_team1,field_goal_percentage_team1,field_goals_made_per_game_team1,...,TeamID_srs2,avg_MOV_team2,total_MOV_team2,games_played_team2,wins_team2,losses_team2,win_pct_team2,SRS_team2,target,optimal_probability
3898,2010,3114,3210,63.0,53.0,Tournament,462,66.843750,44.854155,25.468750,...,3210,8.562500,274,32,23,9,0.718750,16.061312,True,True
4011,2010,3114,3328,60.0,44.0,Tournament,462,66.843750,44.854155,25.468750,...,3328,5.030303,166,33,23,10,0.696970,22.591927,False,False
6081,2010,3122,3397,75.0,42.0,Tournament,-179,66.187500,38.721606,22.906250,...,3397,17.406250,557,32,30,2,0.937500,33.372460,False,False
6516,2010,3124,3163,70.0,50.0,Tournament,488,72.093750,46.311703,26.093750,...,3163,34.757576,1147,33,33,0,1.000000,49.614213,False,False
6534,2010,3124,3181,51.0,48.0,Tournament,488,72.093750,46.311703,26.093750,...,3181,16.531250,529,32,27,5,0.843750,28.913093,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926515,2024,3343,3452,63.0,53.0,Tournament,399,69.724138,45.838084,27.724138,...,3452,16.870968,523,31,24,7,0.774194,28.204894,False,False
928001,2024,3355,3428,68.0,54.0,Tournament,408,73.766667,48.232475,26.833333,...,3428,14.000000,420,30,20,10,0.666667,30.061336,False,False
931729,2024,3394,3425,87.0,55.0,Tournament,152,65.185185,40.090674,22.925926,...,3425,12.516129,388,31,26,5,0.838710,30.939475,False,False
931998,2024,3397,3453,92.0,63.0,Tournament,195,75.935484,43.015633,27.516129,...,3453,16.406250,525,32,26,6,0.812500,15.052416,True,True


# Model Building

The model will be built using XGBoost. The model will be trained on the training data and validated on the validation data. The model will be evaluated using the Brier Score. Due to the unpredictable nature of the tournament, only Tournament Games will be used for training and evaluation. This may be subject to change later.

We will first evaluate the importance of each feature in the model, and then eliminate the least important features. We will then retrain the model and evaluate the Brier Score.

In [18]:
# Check for missing values

print(combined_df_m.isnull().sum())
print(combined_df_w.isnull().sum())

# Print out the column names in full
pd.set_option('display.max_columns', None)
print(combined_df_m.columns)
print(combined_df_w.columns)

# Need to drop columns, such as TeamID, that would complicate the model and unimmportant columns
xgboost_dropped_columns = ['WScore', 'LScore', 'target',
                           'TeamID1', 'TeamID2', 'games_played_team1', 'games_played_team2', 
                           'losses_team1', 'losses_team2', 'wins_team1', 'wins_team2',
                           'TeamID_srs2', 'TeamID', 'field_goals_made_per_game_team1', 'field_goals_made_per_game_team2',
                           'total_MOV_team1', 'total_MOV_team2', 'personal_fouls_per_game_team1', 'personal_fouls_per_game_team2',
                           'total_points_diff_team1', 'total_points_diff_team2', 'steals_per_game_team1', 'steals_per_game_team2',
                           'ConfAbbrev_team1', 'ConfAbbrev_team2', 'points_per_game_team1', 'points_per_game_team2',
                           'turnovers_per_game_team1', 'turnovers_per_game_team2', 'three_point_field_goals_made_per_game_team1', 'three_point_field_goals_made_per_game_team2',
                           'avg_MOV_team1', 'avg_MOV_team2', 'win_pct_team1', 'win_pct_team2',
                           'offensive_rating_team1', 'offensive_rating_team2', 'defensive_rating_team1', 'defensive_rating_team2']

# # Only include data from DataSource = 'Tournament'
# xgboost_data_m = combined_df_m[combined_df_m['DataSource'] == 'Tournament'].drop(columns=xgboost_dropped_columns)

# Drop the columns
xgboost_data_m = combined_df_m.drop(columns=xgboost_dropped_columns)

display(xgboost_data_m)

# For all non-categorical columns, we'll need to find the difference between them and drop the original columns
# This is because supplying both columns to the model would introduce multicollinearity

# Find all non-categorical columns
non_categorical_columns = []
for col in xgboost_data_m.columns:
    if combined_df_m[col].dtype != 'object':
        # Check if column ends with _team1 or _team2
        if col.endswith('_team1') or col.endswith('_team2'):
            # Strip the suffix correctly
            col_short = col[:-6]
            if col_short not in non_categorical_columns:
                non_categorical_columns.append(col_short)

# Find the difference between the columns
for col in non_categorical_columns:
    if col != 'Season':
        xgboost_data_m[f"{col}_diff"] = xgboost_data_m[f"{col}_team1"] - xgboost_data_m[f"{col}_team2"]
        xgboost_data_m.drop(columns=[f"{col}_team1", f"{col}_team2"], inplace=True)

display(xgboost_data_m)


Season           0
TeamID1          0
TeamID2          0
WScore           0
LScore           0
                ..
wins_team2       0
losses_team2     0
win_pct_team2    0
SRS_team2        0
target           0
Length: 63, dtype: int64
Season           0
TeamID1          0
TeamID2          0
WScore           0
LScore           0
                ..
wins_team2       0
losses_team2     0
win_pct_team2    0
SRS_team2        0
target           0
Length: 63, dtype: int64
Index(['Season', 'TeamID1', 'TeamID2', 'WScore', 'LScore', 'DataSource',
       'total_points_diff_team1', 'points_per_game_team1',
       'field_goal_percentage_team1', 'field_goals_made_per_game_team1',
       'three_point_percentage_team1',
       'three_point_field_goals_made_per_game_team1',
       'free_throw_percentage_team1', 'offensive_rebounds_per_game_team1',
       'defensive_rebounds_per_game_team1', 'rebounds_per_game_team1',
       'assists_per_game_team1', 'turnovers_per_game_team1',
       'steals_per_game_tea

,Season,DataSource,field_goal_percentage_team1,three_point_percentage_team1,free_throw_percentage_team1,offensive_rebounds_per_game_team1,defensive_rebounds_per_game_team1,rebounds_per_game_team1,assists_per_game_team1,blocks_per_game_team1,net_rating_team1,seed_team1,field_goal_percentage_team2,three_point_percentage_team2,free_throw_percentage_team2,offensive_rebounds_per_game_team2,defensive_rebounds_per_game_team2,rebounds_per_game_team2,assists_per_game_team2,blocks_per_game_team2,net_rating_team2,seed_team2,SRS_team1,SRS_team2
0,2003,Other,48.114901,37.564322,65.135699,4.178571,16.821429,21.000000,13.000000,1.785714,-0.138751,16.0,48.607427,33.870968,73.638968,9.777778,19.925926,29.703704,15.222222,2.333333,0.874474,16.0,2.266120,-0.416138
1,2003,Other,48.114901,37.564322,65.135699,4.178571,16.821429,21.000000,13.000000,1.785714,-0.138751,16.0,42.036227,32.014388,70.989761,13.571429,23.928571,37.500000,12.107143,3.785714,6.607376,10.0,2.266120,14.222003
2,2003,Other,48.114901,37.564322,65.135699,4.178571,16.821429,21.000000,13.000000,1.785714,-0.138751,16.0,39.575531,36.481481,70.598592,13.500000,23.115385,36.615385,14.538462,2.076923,-8.082866,16.0,2.266120,-16.032281
3,2003,Other,48.114901,37.564322,65.135699,4.178571,16.821429,21.000000,13.000000,1.785714,-0.138751,16.0,42.377261,34.615385,64.642082,12.285714,23.857143,36.142857,11.678571,3.142857,-0.919782,16.0,2.266120,-9.467136
4,2003,Other,48.114901,37.564322,65.135699,4.178571,16.821429,21.000000,13.000000,1.785714,-0.138751,16.0,41.827222,35.748792,73.350923,8.250000,20.250000,28.500000,11.928571,2.035714,-13.741472,16.0,2.266120,-14.606844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416767,2025,Other,41.598600,31.400966,64.375000,7.935484,19.000000,26.935484,14.161290,3.903226,-16.617704,16.0,42.136695,35.701275,80.686695,6.214286,18.214286,24.428571,13.107143,1.892857,-8.570939,16.0,-12.234262,-15.068266
1416768,2025,Other,41.598600,31.400966,64.375000,7.935484,19.000000,26.935484,14.161290,3.903226,-16.617704,16.0,42.720630,29.664179,69.005848,8.166667,21.166667,29.333333,12.966667,2.966667,-16.713512,16.0,-12.234262,-14.876635
1416769,2025,RegularSeason,44.623990,33.034380,71.735537,7.448276,21.758621,29.206897,13.517241,2.103448,-13.725047,16.0,42.136695,35.701275,80.686695,6.214286,18.214286,24.428571,13.107143,1.892857,-8.570939,16.0,-17.121246,-15.068266
1416771,2025,Other,44.623990,33.034380,71.735537,7.448276,21.758621,29.206897,13.517241,2.103448,-13.725047,16.0,42.720630,29.664179,69.005848,8.166667,21.166667,29.333333,12.966667,2.966667,-16.713512,16.0,-17.121246,-14.876635


,Season,DataSource,field_goal_percentage_diff,three_point_percentage_diff,free_throw_percentage_diff,offensive_rebounds_per_game_diff,defensive_rebounds_per_game_diff,rebounds_per_game_diff,assists_per_game_diff,blocks_per_game_diff,net_rating_diff,seed_diff,SRS_diff
0,2003,Other,-0.492526,3.693355,-8.503269,-5.599206,-3.104497,-8.703704,-2.222222,-0.547619,-1.013225,0.0,2.682258
1,2003,Other,6.078674,5.549934,-5.854062,-9.392857,-7.107143,-16.500000,0.892857,-2.000000,-6.746127,6.0,-11.955884
2,2003,Other,8.539371,1.082841,-5.462892,-9.321429,-6.293956,-15.615385,-1.538462,-0.291209,7.944115,0.0,18.298401
3,2003,Other,5.737640,2.948938,0.493617,-8.107143,-7.035714,-15.142857,1.321429,-1.357143,0.781031,0.0,11.733255
4,2003,Other,6.287679,1.815530,-8.215224,-4.071429,-3.428571,-7.500000,1.071429,-0.250000,13.602720,0.0,16.872963
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416767,2025,Other,-0.538096,-4.300309,-16.311695,1.721198,0.785714,2.506912,1.054147,2.010369,-8.046765,0.0,2.834004
1416768,2025,Other,-1.122030,1.736787,-4.630848,-0.231183,-2.166667,-2.397849,1.194624,0.936559,0.095808,0.0,2.642374
1416769,2025,RegularSeason,2.487295,-2.666895,-8.951158,1.233990,3.544335,4.778325,0.410099,0.210591,-5.154108,0.0,-2.052980
1416771,2025,Other,1.903360,3.370201,2.729689,-0.718391,0.591954,-0.126437,0.550575,-0.863218,2.988465,0.0,-2.244611


In [19]:



# # Save the ConfAbbrev for both teams
# confabbrev_team1 = xgboost_data_m['ConfAbbrev_team1']
# confabbrev_team2 = xgboost_data_m['ConfAbbrev_team2']

# # Ensure ConfAbbrev_team1 and ConfAbbrev_team2 are categorical, so that XGBoost can handle them properly
# xgboost_data_m['ConfAbbrev_team1'] = pd.Categorical(xgboost_data_m['ConfAbbrev_team1']).codes
# xgboost_data_m['ConfAbbrev_team2'] = pd.Categorical(xgboost_data_m['ConfAbbrev_team2']).codes

# # Print out the codes for each conference abbreviation so we can understand the mapping
# print("Here's a mapping of the original conference abbreviations to the encoded values:")
# conf_mapping = pd.DataFrame({
#     'ConfAbbrev': confabbrev_team1.unique(),
#     'Code': xgboost_data_m['ConfAbbrev_team1'].unique()
# })
# print(conf_mapping)

# Setup XGBoost to predict target, which is the probability that TeamID1 wins
# We'll need to drop the 2025 season, as that's the season we're predicting

xgboost_data_m = xgboost_data_m[xgboost_data_m['Season'] != 2025]

# Split the data into training and testing sets
X = xgboost_data_m.drop(columns=['Season'])
y = combined_df_m.loc[xgboost_data_m.index, 'target']

# Ensure the target variable is binary
y = y.astype(int)

# Use GroupShuffleSplit to split by Season
group_split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(group_split.split(X, y, groups=xgboost_data_m['Season']))

# Split the data
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Drop the non-tournament data from the _test data
X_test = X_test[X_test.index.isin(xgboost_data_m[xgboost_data_m['DataSource'] == 'Tournament'].index)]
y_test = y_test[y_test.index.isin(xgboost_data_m[xgboost_data_m['DataSource'] == 'Tournament'].index)]

# Drop the DataSource column
X_train.drop(columns=['DataSource'], inplace=True)
X_test.drop(columns=['DataSource'], inplace=True)
y_train.drop(columns=['DataSource'], inplace=True)
y_test.drop(columns=['DataSource'], inplace=True)

# Verify that seasons are not split
print("Training set seasons:", xgboost_data_m.iloc[train_idx]['Season'].unique())
print("Testing set seasons:", xgboost_data_m.iloc[test_idx]['Season'].unique())

# Create the XGBoost model, with the evaluation method as the Brier Score.
# The Brier Score is a measure of the mean squared error of the predicted probabilities.
# The target variable is a float with range [0,1], so we can use the 'binary:logistic' objective function
# The vars that have the suffix _team1 or _team2 are the team statistics for each team

import matplotlib.pyplot as plt
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, brier_score_loss
import os

# Define the XGBoost model with proper handling of categorical features
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    enable_categorical=True,
    random_state=42  # For reproducibility
)

# Define hyperparameters for GridSearchCV
param_grid = {
    'n_estimators': [300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.005, 0.01],
    'subsample': [0.5, 0.6, 0.7],
    'colsample_bytree': [0.8, 1.0]
}

# Use GridSearchCV for hyperparameter tuning
# Get the number of available CPU cores
total_cores = os.cpu_count()
# Calculate cores to use (leave half unallocated
cores_to_use = max(1, total_cores // 2)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_brier_score',
    cv=5,
    n_jobs=cores_to_use,  # Use all cores except 4
    verbose=1,
    error_score='raise'
)

# Fit the model
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters:", grid_search.best_params_)
print("Best score:", -grid_search.best_score_)  # Negate to get actual Brier score

# Evaluate on test set
y_pred_proba = grid_search.predict_proba(X_test)[:, 1]
brier_score = brier_score_loss(y_test, y_pred_proba)
print(f"Test set Brier score: {brier_score:.4f}")

# Get the best model
best_model = grid_search.best_estimator_

# Get feature importance
importance = best_model.get_booster().get_score(importance_type='gain')

# Convert to a DataFrame for easier analysis
importance_df = pd.DataFrame({
    'Feature': list(importance.keys()),
    'Importance': list(importance.values())
}).sort_values(by='Importance', ascending=False)

# Save the importance_df
importance_df_dir = r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\March_Madness_2025_Kaggle\Feature_Importance"
importance_df_path = os.path.join(importance_df_dir, 'importance_df_v6.csv')
importance_df.to_csv(importance_df_path, index=False)

# Print the top 10 most important features
print(importance_df.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
plot_importance(best_model, importance_type='gain', max_num_features=20, height=0.8)
plt.title('Feature Importance (Gain)')

# Save the figure
importance_plot_dir = r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\March_Madness_2025_Kaggle\Feature_Importance"
importance_plot_path = os.path.join(importance_plot_dir, 'importance_plot_v6.png')
plt.savefig(importance_plot_path)

plt.show()



# Save the best model
best_model_dir = r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\March_Madness_2025_Kaggle\Models"
best_model_path = os.path.join(best_model_dir, 'best_model_v6')
best_model.save_model(best_model_path)



C:\Users\henry-cao-local\AppData\Local\Temp\ipykernel_9188\3074457401.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=['DataSource'], inplace=True)


Training set seasons: [2005 2006 2007 2008 2009 2010 2012 2013 2014 2015 2017 2019 2020 2021
 2022 2023 2024]
Testing set seasons: [2003 2004 2011 2016 2018]
Fitting 5 folds for each of 36 candidates, totalling 180 fits


KeyboardInterrupt: 

In [ ]:
# Make predictions
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1] 

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
brier_score = brier_score_loss(y_test, y_pred_proba)

# Print results
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Brier Score: {brier_score:.4f}")

# Predict for 2025 season
xgboost_data_m_2025 = xgboost_data_m[xgboost_data_m['Season'] == 2025].drop(columns=['Season'])

# Make predictions
y_pred_2025 = best_model.predict(xgboost_data_m_2025)
y_pred_proba_2025 = best_model.predict_proba(xgboost_data_m_2025)[:, 1]

# Save the 2025 predictions to a csv file
output_2025_dir = r"C:\Users\henry-cao-local\Desktop\Self_Learning\Kaggle\March_Machine_Learning_Mania_2025\March_Madness_2025_Kaggle\Predictions"
output_2025_path = os.path.join(output_2025_dir, 'predictions_2025_v1.csv')

# Create a DataFrame with the predictions
y_pred_2025_df = pd.DataFrame({
    'Season': 2025,
    'TeamID1': xgboost_data_m_2025['TeamID1'],
    'TeamID2': xgboost_data_m_2025['TeamID2'],
    'Predicted_Probability': y_pred_proba_2025
})

# Save the predictions to a csv file
y_pred_2025_df.to_csv(output_2025_path, index=False)